In [2]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e

dic = {
    'gene':'./data/gene/GRCh37.gene.bed',
    'non_gene':'./data/gene/non_gene.bed',
    'fasta':'/home/jiay/Desktop/hg19/hg19.fa',
    'bam1':'./data/051.bam',
    'bam2':'./data/051_2.bam',
    'bam3':'./data/051_3.bam',
    'bam123':'./data/051_123.bam',
    'TSS_low':'./data/gene/low_expressed.bed',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed'
    }
chr_to_id = {
    'chr1':'1','chr2':'2','chr3':'3','chr4':'4','chr5':'5','chr6':'6','chr7':'7','chr8':'8','chr9':'9','chr10':'10','chr11':'11','chr12':'12','chr13':'13','chr14':'14','chr15':'15','chr16':'16','chr17':'17','chr18':'18','chr19':'19','chr20':'20','chr21':'21','chr22':'22','chrX':'X','chrY':'Y'
}
up = 1000
down = 1000

fasta = ps.FastaFile(dic['fasta'])
bam1file = ps.AlignmentFile(dic['bam1'],'rb')
gene = []
with open(dic['gene'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['1']:
            gene.append([ll[0],int(ll[1]),int(ll[2])])
gene = sorted(gene, key= lambda x: x[1])
non_gene = []

for i, gg in enumerate(gene):
    if i == 0 or i == len(gene) - 1:
        continue
    start = gg[1] - 4000
    end = gg[1] - 2000
    if start > gene[i-1][2]:
        if 'N' not in fasta.fetch('chr1',start,end):
            non_gene.append(['1', start, end])
    start = gg[2] + 2000
    end = gg[2] + 4000
    if end < gene[i+1][1]:
        if 'N' not in fasta.fetch('chr1',start,end):
            non_gene.append(['1', start, end])
CCR_coverage = np.zeros((len(non_gene),2000),dtype = int)

CCR_long_array = np.zeros((len(non_gene), 2000), dtype=int)
CCR_short_array = np.zeros((len(non_gene), 2000), dtype=int)

for i, region in enumerate(non_gene):
    chrom = region[0]
    start = region[1]
    end = region[2]
    for r in bam1file.fetch(chrom, start-500, end+500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            ss = max(0, r.reference_start - start)
            ee = min(r.reference_start - start + abs(r.isize), end - start)
            for p in range(ss, ee):
                CCR_coverage[i][p] += 1
            if 200 >= abs(r.isize) > 130:
                for p in range(ss, ee):
                    CCR_long_array[i][p] += 1
            if abs(r.isize) <= 130:
                for p in range(ss, ee):
                    CCR_short_array[i][p] += 1
    
TSS_low = []
with open(dic['TSS_low'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['chr1','chr2','chr3']:
            TSS_low.append(TSS(chr_to_id[ll[0]],int(ll[1])))

TSS_HK1 = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '1':
            TSS_HK1.append(TSS(ll[0], int(ll[1])+int(1000)))
TSS_HK2 = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '2':
            TSS_HK2.append(TSS(ll[0], int(ll[1])+int(1000)))
TSS_HK3 = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '3':
            TSS_HK3.append(TSS(ll[0], int(ll[1])+int(1000)))

HK_long_array = np.zeros((len(TSS_HK1), int(up+down)), dtype=int)
HK_short_array = np.zeros((len(TSS_HK1), int(up+down)), dtype=int)
HK_coverage = np.zeros((len(TSS_HK1),int(up+down)), dtype =int)
for j, tss in enumerate(TSS_HK1):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    for r in bam1file.fetch(chrom, start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            ss = max(0, r.reference_start - start)
            ee = min(r.reference_start - start + abs(r.isize), end - start)
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    HK_short_array[j][i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    HK_long_array[j][i] += 1
            for i in range(ss, ee):
                HK_coverage[j][i] += 1


In [3]:
# CCR_coverage = np.zeros((len(non_gene),int(up+down)), dtype= float)
CCR_x_di_y = np.zeros((len(non_gene),int(up+down)), dtype= float)
for i in range(len(non_gene)):
    for j in range((up+down)):
        # CCR_coverage[i][j] = float(CCR_long_array[i][j]) + float(CCR_short_array[i][j])
        CCR_x_di_y[i][j] = float(CCR_long_array[i][j]) - float(CCR_short_array[i][j])
# HK_coverage = np.zeros((len(TSS_HK1),int(up+down)), dtype= float)
HK_x_di_y = np.zeros((len(TSS_HK1),int(up+down)), dtype= float)
for i in range(len(TSS_HK1)):
    for j in range((up+down)):
        # HK_coverage[i][j] = float(HK_long_array[i][j]) + float(HK_short_array[i][j])
        HK_x_di_y[i][j] = float(HK_long_array[i][j]) - float(HK_short_array[i][j])

In [6]:
print(len(TSS_HK1),len(TSS_HK2),len(TSS_HK3))

369 235 215


In [4]:
bam123file = ps.AlignmentFile(dic['bam123'],'rb')
test_gene = []
with open(dic['gene'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['2','3']:
            test_gene.append([ll[0],int(ll[1]),int(ll[2])])
test_gene = sorted(test_gene, key= lambda x: x[1])
test_non_gene = []

for i, gg in enumerate(test_gene):
    if i == 0 or i == len(test_gene) - 1:
        continue
    start = gg[1] - 4000
    end = gg[1] - 2000
    if start > test_gene[i-1][2]:
        if 'N' not in fasta.fetch('chr'+gg[0],start,end):
            test_non_gene.append([gg[0], start, end])
    start = gg[2] + 2000
    end = gg[2] + 4000
    if end < test_gene[i+1][1]:
        if 'N' not in fasta.fetch('chr'+gg[0],start,end):
            test_non_gene.append([gg[0], start, end])
test_CCR_long_array = np.zeros((len(test_non_gene), 2000), dtype=int)
test_CCR_short_array = np.zeros((len(test_non_gene), 2000), dtype=int)
test_CCR_coverage = np.zeros((len(test_non_gene),int(up+down)), dtype= float)
for i, region in enumerate(test_non_gene):
    chrom = region[0]
    start = region[1]
    end = region[2]
    for r in bam123file.fetch(chrom, start-500, end+500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            ss = max(0, r.reference_start - start)
            ee = min(r.reference_start - start + abs(r.isize), end - start)
            if 200 >= abs(r.isize) > 130:
                for p in range(ss, ee):
                    test_CCR_long_array[i][p] += 1
            if abs(r.isize) <= 130:
                for p in range(ss, ee):
                    test_CCR_short_array[i][p] += 1
            for p in range(ss, ee):
                test_CCR_coverage[i][p] += 1

test_HK_long_array = np.zeros((len(TSS_HK2+TSS_HK3), int(up+down)), dtype=int)
test_HK_short_array = np.zeros((len(TSS_HK2+TSS_HK3), int(up+down)), dtype=int)
test_HK_coverage = np.zeros((len(TSS_HK2+TSS_HK3),int(up+down)), dtype= float)
for j, tss in enumerate(TSS_HK2+TSS_HK3):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    for r in bam123file.fetch(chrom, start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            ss = max(0, r.reference_start - start)
            ee = min(r.reference_start - start + abs(r.isize), end - start)
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    test_HK_short_array[j][i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    test_HK_long_array[j][i] += 1
            for i in range(ss, ee):
                test_HK_coverage[j][i] += 1
# test_CCR_coverage = np.zeros((len(test_non_gene),int(up+down)), dtype= float)
test_CCR_x_di_y = np.zeros((len(test_non_gene),int(up+down)), dtype= float)
for i in range(len(test_non_gene)):
    for j in range((up+down)):
        # test_CCR_coverage[i][j] = float(test_CCR_long_array[i][j]) + float(test_CCR_short_array[i][j])
        test_CCR_x_di_y[i][j] = float(test_CCR_long_array[i][j]) - float(test_CCR_short_array[i][j])
# test_HK_coverage = np.zeros((len(TSS_HK2+TSS_HK3),int(up+down)), dtype= float)
test_HK_x_di_y = np.zeros((len(TSS_HK2+TSS_HK3),int(up+down)), dtype= float)
for i in range(len(TSS_HK2+TSS_HK3)):
    for j in range((up+down)):
        # test_HK_coverage[i][j] = float(test_HK_long_array[i][j]) + float(test_HK_short_array[i][j])
        test_HK_x_di_y[i][j] = float(test_HK_long_array[i][j]) - float(test_HK_short_array[i][j])

In [9]:
'''
svm
'''
import sklearn
model = sklearn.svm.SVC(kernel='linear', C=1, gamma=1)
train_x = []
train_y = []
for i, cov in enumerate(HK_coverage):
    # train_x.append([sum(cov[850:1150])])
    train_x.append([sum(HK_x_di_y[i][850:1150])])
    train_y.append(1)
for i, cov in enumerate(CCR_coverage[:len(HK_coverage)]):
    # train_x.append([sum(cov[850:1150])])
    train_x.append([sum(CCR_x_di_y[i][850:1150])])
    train_y.append(0)
model.fit(train_x,train_y)

test_x = []
test_y = []
for i, cov in enumerate(test_HK_coverage):
    # test_x.append([sum(cov[850:1150])])
    test_x.append([sum(test_HK_x_di_y[i][850:1150])])
    test_y.append(1)
for i, cov in enumerate(test_CCR_coverage[:len(test_HK_coverage)]):
    # test_x.append([sum(cov[850:1150])])
    test_x.append([sum(test_CCR_x_di_y[i][850:1150])])
    test_y.append(0)

pred_y = model.predict(test_x)
[[TN, FP],[FN, TP]] = sklearn.metrics.confusion_matrix(test_y,pred_y)
cc = sklearn.metrics.confusion_matrix(test_y,pred_y)
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
F1_score = 2 * Precision * Recall / (Precision + Recall)
Accuracy = (TP + TN) / (TP + TN + FP + FN)
print(Precision, Recall, F1_score, Accuracy)
print(cc)

0.9404466501240695 0.8422222222222222 0.8886283704572098 0.8944444444444445
[[426  24]
 [ 71 379]]


0.7846481876332623 0.8177777777777778 0.8008705114254624 0.9509777658719528
[[3182  101]
 [  82  368]]

0.6182707993474714 0.8422222222222222 0.7130761994355597 0.9182962764532547
[[3049  234]
 [  71  379]]

In [8]:
'''
lstm
'''

HK_win_coverage = []
win_size = 200
for cov in test_HK_coverage:
    win = []
    k = 0
    while True:
        ss = int(k)
        ee = ss + win_size
        k = k + win_size / 2
        win.append(np.mean(cov[ss:ee]))
        if ee >= up+down:
            break
    HK_win_coverage.append(win)
CCR_win_coverage = []
win_size = 200
for cov in test_CCR_coverage:
    win = []
    k = 0
    while True:
        ss = int(k)
        ee = ss + win_size
        k = k + win_size / 2
        win.append(np.mean(cov[ss:ee]))
        if ee >= up+down:
            break
    CCR_win_coverage.append(win)

with open('./test_HK.bed','w') as f:
    for i, win in enumerate(HK_win_coverage):
        for j, cov in enumerate(win): 
            f.write(str(cov)+'\t')
        f.write('\n')
# from keras.models import Sequential
# from keras import layers
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras import optimizers
# import tensorflow as tf

